In [323]:
%reset -f

In [623]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.special import eval_chebyt
from numpy.polynomial.chebyshev import Chebyshev
from numpy.polynomial.chebyshev import chebval, chebder

# Параметры
tau, h, T = 0.5, 0.1, 70.0
num_exp = 100 # Число траекторий
t = np.arange(0, T + h, h)
N = len(t)
M = int(tau / h)
m = 2

lambda_test = [-4,-14,-16.5]
mu_test = [1, 2, 3]
omega_test = 0.5

def f1(y):
    return lambda_test[0] * y  
def f2(y):
    return lambda_test[1] * y
def f3(y):
    return lambda_test[2] * y

def g1(y, yd):
    return [mu_test[0]*y + omega_test*yd for _ in range(m)]
def g2(y, yd):
    return [mu_test[1]*y + omega_test*yd for _ in range(m)]
def g3(y, yd):
    return [mu_test[2]*y + omega_test*yd for _ in range(m)]


def y0(t):
    return np.ones_like(t)

In [625]:
def __Levi_integral(dw, h, m, num, seed=None):
    """Интеграл Леви для вычисления аппроксимации интеграла Ито
    num --- число членов ряда"""
    if seed != None:
        np.random.seed(seed)
        
    X = np.random.normal(loc=0.0, scale=1.0, size=(m, num))
    Y = np.random.normal(loc=0.0, scale=1.0, size=(m, num))

    K = range(0, num, 1)

    A1 = [np.outer(X[:, k], (Y[:, k] + np.sqrt(2.0/h)*dw)) for k in K]
    A2 = [np.outer((Y[:, k] + np.sqrt(2.0/h)*dw), X[:, k]) for k in K]
    A = ((1.0/(k+1))*(A1[k] - A2[k]) for k in K)
    A = (h/np.pi)*np.sum(A)

    return A


def Ito1Wm(dW, h):
    """Генерирование значений однократного интеграла Ито для многомерного
    винеровского процесса"""
    (N, m) = dW.shape
    Ito = np.empty(shape=(N, m+1))
    Ito[:, 0] = h
    Ito[:, 1:] = dW[:, :]
    return Ito


def Ito2Wm(dW, h, n, seed=None):
    """Вычисление интеграла Ито (по точной формуле или по аппроксимации)
    Возвращает список матриц I(i,j)"""
    (N, m) = dW.shape
    E = np.identity(m)
    Ito = np.empty(shape=(N, m+1, m+1))
    if seed != None:
        np.random.seed(seed)
    dzeta = np.random.normal(loc=0, scale=h, size=(N, m))
    Ito[:, 0, 0] = h
    for dw, I, dz in zip(dW, Ito, dzeta):
        I[0, 1:] = 0.5*h*(dw - dz / np.sqrt(3))
        I[1:, 0] = 0.5*h*(dw + dz / np.sqrt(3))
        I[1:, 1:] = 0.5*(np.outer(dw, dw) - h*E) + __Levi_integral(dw, h, m, n, seed+1)
    return Ito

In [627]:
def chebyt_derivative(n, x, eps=1e-6):
    """Численная производная T_n(x) (можно заменить аналитической формулой)"""
    return (eval_chebyt(n, x + eps) - eval_chebyt(n, x - eps)) / (2 * eps)

In [629]:
def srockd1_sdde(f, g, y0, T, tau, M, dW, I1, I2, h, N, m, s, eta, beta1, beta2, B, A, seed):         
    y = np.zeros(N+1)
    t_history = np.linspace(-tau, 0, M+1)
    y[:M+1] = y0(t_history)
    t = np.linspace(0, T, N + 1)
    for n in range(N):
        
        y_curr = y[n]
        y_prev = y[n - M] if n-M >= 0 else y0(t[n] - tau)

        K = np.zeros(s+1)
        omega0 = 1 + eta/ s**2
        Ts_omega0 = eval_chebyt(s, omega0)
        Ts_prime_omega0 = chebyt_derivative(s, omega0)  # Производная T_s(omega0)
        omega1 = Ts_omega0 / Ts_prime_omega0
        K[0] = y_curr
        K[1] = y_curr + h * (omega1 / omega0) * f(K[0])
        for j in range(2, s+1):
            T_j = eval_chebyt(j, omega0)
            T_jm1 = eval_chebyt(j-1, omega0)
            T_jm2 = eval_chebyt(j-2, omega0)
            K[j] = (2 * T_jm1 / T_j) * (h * omega1 * f(K[j-1])) + (2 * omega0 * T_jm1 / T_j) * K[j-1] - (T_jm2 / T_j) * K[j-2]
        
        H = np.zeros((s, m))
        for v in range(m):  
            for i in range(s):
                det_part = y_curr
                temp_det = 0
                for k in range(1,s+1):
                    temp_det+=A[i, k-1] * h * f(K[k-1])
                # Детерминированная часть
                det_part += temp_det
                # Стохастическая часть 
                stoch_part = 0.0
                for k in range(i):  # Сумма по i_b (k < i)
                    for l in range(m):
                        gl = g(H[i,l], y_prev)[l]
                        stoch_part += B[i, k] * gl * I2[n][l+1][v+1]  # I2[l][j] = ζ̃^(l,j)
                H[i][v] = det_part + stoch_part
        # Обновление решения
        y[n+1] = K[s]
        stoch_term = 0
        for i in range(s):
            for v in range(m):
                gv = g(H[i, v], y_prev)[v]
                stoch_term += beta1[i] * gv * I1[n, v+1]  # ΔW_j
                stoch_term += beta2[i] * gv * np.sqrt(h)  # √h·ζ 
        y[n+1] += stoch_term
        
    return y[:N]

In [631]:
def compute_alphas(s, eta):
    x0 = 1 + eta / s**2
    
    # Коэффициенты полиномов Чебышёва
    c_Ts = np.zeros(s+1)
    c_Ts[s] = 1  # T_s(x)
    
    c_Ts_der = chebder(c_Ts)  # T_s'(x)
    c_Ts1 = np.zeros(s)
    c_Ts1[s-1] = 1  # T_{s-1}(x)
    
    # Вычисление значений в точке x0
    Ts_x0 = chebval(x0, c_Ts)
    Ts_der_x0 = chebval(x0, c_Ts_der)
    Ts1_x0 = chebval(x0, c_Ts1)
    
    alphas = np.zeros(s)
    for i in range(1, s):  # Только до s-1!
        c_Ti1 = np.zeros(i)
        if i > 0:
            c_Ti1[i-1] = 1  # T_{i-1}(x)
        Ti1_x0 = chebval(x0, c_Ti1)
        
        alphas[i-1] = (2/s) * (Ts1_x0/Ts_der_x0) * (Ti1_x0/Ts_x0)
    
    alphas = alphas / np.sum(alphas)
    return alphas

In [633]:
s = 3
eta = 0.5

B = np.array([[0, 0, 0], [1, 0, 0], [-1, 0, 0]])  # Параметры B
beta1 = np.array([1, 0, 0])  # Параметры β1
beta2 = np.array([0, 0.5, -0.5])  # Параметры β2

In [635]:
alphas = compute_alphas(s, eta)
A = np.zeros((s,s))
for i in range(s):
    for j in range(s):
            A[i][j] = alphas[j]


In [637]:
# Массивы для данных
y1 = np.zeros((num_exp, N))
y2 = np.zeros((num_exp, N))
y3 = np.zeros((num_exp, N))
seed1 = 42
seed2 = 43
seed3 = 44

In [ ]:
for j in range(num_exp):
    dW1 = np.sqrt(h) * np.random.normal(size=(N, m))
    dW2 = np.sqrt(h) * np.random.normal(size=(N, m))
    dW3 = np.sqrt(h) * np.random.normal(size=(N, m))
    I11 = Ito1Wm(dW1, h)
    I21 = Ito2Wm(dW1, h, 100, seed = 42)
    I12 = Ito1Wm(dW2, h)
    I22 = Ito2Wm(dW2, h, 100, seed = 43)
    I13 = Ito1Wm(dW3, h)
    I23 = Ito2Wm(dW3, h, 100, seed = 44)
    y1[j] = srockd1_sdde(f1, g1, y0, T, tau, M, dW1, I11, I21, h, N, m, s, eta, beta1, beta2, B, A, seed1)
    y2[j] = srockd1_sdde(f2, g2, y0, T, tau, M, dW2, I12, I22, h, N, m, s, eta, beta1, beta2, B, A, seed2)
    y3[j] = srockd1_sdde(f3, g3, y0, T, tau, M, dW3, I13, I23, h, N, m, s, eta, beta1, beta2, B, A, seed3)
    print(j)
# Усреднение
E1_y2 = np.mean(np.abs(y1)**2, axis=0)
E2_y2 = np.mean(np.abs(y2)**2, axis=0)
E3_y2 = np.mean(np.abs(y3)**2, axis=0)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(t, E1_y2, label=r'$p = (-4.5,0), \; q = 1, \; r = 0.5$', color='blue')
plt.plot(t, E2_y2, label=r'$p = (-14,0.5), \; q = 2, \; r = 0.5$', color='red')
plt.plot(t, E3_y2, label=r'$p = (-16.5,0.6), \; q = 3, \; r = 0.5$', color='green')
plt.yscale('log')  # Логарифмическая шкала для оси Y
plt.xlabel('Время $t$', fontsize=12)
plt.ylabel(r'$\mathbb{E}[|y_n|^2]$', fontsize=12)
plt.title('MS-устойчивость решения S-ROCK1 методом', fontsize=14)
plt.grid(True, linestyle='--', which='both')  # Сетка для логарифмической шкалы
plt.legend()
plt.savefig("MS322", dpi=300, bbox_inches='tight')
plt.show()